In [3]:
import os
import requests
from bs4 import BeautifulSoup
import csv
import re

# url base de la tienda online
url="https://latiendadevalentina.com"


# robots.txt

def check_robots_txt(url):
  url_robots = url + '/robots.txt'
  page = requests.get(url_robots)
  soup = BeautifulSoup(page.content, 'lxml')
  print(soup.prettify())
  
  
check_robots_txt(url) 
# vemos como el fichero robots.txt presenta restricciones principalmente relacionadas
# con la tecnología que se encarga de gestionar la tienda y el pago online (Prestashop)
# En principio, no debería afectar al scraper que obtenga los datos de esta práctica
  

<html>
 <body>
  <p>
   # robots.txt automatically generated by PrestaShop e-commerce open-source solution
# http://www.prestashop.com - http://www.prestashop.com/forums
# This file is to prevent the crawling and indexing of certain parts
# of your site by web crawlers and spiders run by sites like Yahoo!
# and Google. By telling these "robots" where not to go on your site,
# you save bandwidth and server resources.
# For more information about the robots.txt standard, see:
# http://www.robotstxt.org/robotstxt.html
User-agent: *
# Allow Directives
Allow: */modules/*.css
Allow: */modules/*.js
Allow: */modules/*.png
Allow: */modules/*.jpg
Allow: */themes/*/assets/cache/*.js
Allow: */themes/*/assets/cache/*.css
Allow: */themes/*/assets/css/*
# Private pages
Disallow: /*?order=
Disallow: /*?tag=
Disallow: /*?id_currency=
Disallow: /*?search_query=
Disallow: /*?back=
Disallow: /*?n=
Disallow: /*&amp;order=
Disallow: /*&amp;tag=
Disallow: /*&amp;id_currency=
Disallow: /*&amp;search_query=
Di

In [1]:
# request a la página principal del sitio
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

menu = soup.find("ul", {"id":"menu"})
# creamos un diccionario que guarda la seccion a la que pertenece cada URL
dict_url_section={}

for li in menu.find_all("li")[2:]: # eliminamos el 1er elemento (relativo al menu) y el 2do (relativo a looks, donde no hay productos)
  href = li.a.attrs['href'] # URL
  name = li.a.attrs['title'] # nombre de la seccion
 
  # algunos href se presentan en la forma /faldas 
  # para esos casos, añadimos la URL base antes, para crear la URL de la seccion
  if re.match(r"https.*", href): 
    dict_url_section[href]=name
  else:
    url = "https://latiendadevalentina.com" + href
    dict_url_section[url]=name
    
print(dict_url_section)    

{'https://latiendadevalentina.com/moda': 'COLECCIÓN', 'https://latiendadevalentina.com/moda-nueva-coleccion': 'NOVEDADES', 'https://latiendadevalentina.com/moda-outlet': 'Outlet', 'https://latiendadevalentina.com/vestidos': 'Vestidos', 'https://latiendadevalentina.com/faldas': 'Faldas', 'https://latiendadevalentina.com/pantalones': 'Pantalones', 'https://latiendadevalentina.com/jeans': 'Jeans', 'https://latiendadevalentina.com/chaquetas': 'Chaquetas y Rebecas', 'https://latiendadevalentina.com/camisetas-y-camisas': 'Camisetas y Camisas', 'https://latiendadevalentina.com/jerseys': 'Jerseis', 'https://latiendadevalentina.com/moda?q=Temporada-Top+Ventas': '¡Top Ventas!', 'https://latiendadevalentina.com/basicos': 'Básicos', 'https://latiendadevalentina.com/zapatos': 'Colección', 'https://latiendadevalentina.com/zapatos-nueva-coleccion': 'Novedades', 'https://latiendadevalentina.com/zapatos-outlet': 'Outlet', 'https://latiendadevalentina.com/botas-y-botines': 'Botas y Botines', 'https://la

In [13]:
# secciones: dataset con las secciones del sitio
# creamos section list y dict pages



section_list = []

# nombre de las variables
section_col_names = ['Name', "N_items", "N_pages"]
section_list.append(section_col_names)

# creamos un diccionario que para cada sección nos indique cuantas páginas tiene
dict_pages={}


# para cada sección, obtenemos los datos de interés
for url,sect in dict_url_section.items():
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'lxml')
  div = soup.find("div",{"class":"col-md-3 hidden-sm-down total-products hide-mobile"})
  section = [sect]
  
  # excepción que se ocupa de los casos en los que no hay el mensaje con el número de items
  try:
    n_items = re.findall(r'\d+',div.p.get_text())[0]
    n_items = int(float(n_items))
    section.append(n_items)
  except:
    n_items = 1   
    section.append(n_items)
  
  # excepción que se ocupa de los casos en los que solo hay una página para navegar en la sección
  try:
       
    soup2 = soup.find("ul",{"class": "page-list clearfix text-sm-center"})
    n_pages = soup2.find_all("a")[-2].get_text().strip()
    n_pages = int(float(n_pages))

    section.append(n_pages)
    
  except:
   
    n_pages = 1
    section.append(n_pages)
    
  section_list.append(section)  
  dict_pages[url]=n_pages

print(dict_pages)

{'https://latiendadevalentina.com/moda': 6, 'https://latiendadevalentina.com/moda-nueva-coleccion': 4, 'https://latiendadevalentina.com/moda-outlet': 6, 'https://latiendadevalentina.com/vestidos': 2, 'https://latiendadevalentina.com/faldas': 1, 'https://latiendadevalentina.com/pantalones': 1, 'https://latiendadevalentina.com/jeans': 1, 'https://latiendadevalentina.com/chaquetas': 1, 'https://latiendadevalentina.com/camisetas-y-camisas': 1, 'https://latiendadevalentina.com/jerseys': 1, 'https://latiendadevalentina.com/moda?q=Temporada-Top+Ventas': 1, 'https://latiendadevalentina.com/basicos': 2, 'https://latiendadevalentina.com/zapatos': 2, 'https://latiendadevalentina.com/zapatos-nueva-coleccion': 1, 'https://latiendadevalentina.com/zapatos-outlet': 1, 'https://latiendadevalentina.com/botas-y-botines': 1, 'https://latiendadevalentina.com/deportivos': 2, 'https://latiendadevalentina.com/sandalias-tacon?q=Familia-Zapatos/Tipo+Prenda-Sandalias+Tacón': 1, 'https://latiendadevalentina.com/s

In [14]:
## construimos un dataframe con los datos obtenidos: secciones

import pandas as pd
df = pd.DataFrame(section_list)

col_names = df.iloc[0]
sections_df  = pd.DataFrame(df.values[1:], columns = col_names)

sections_df

,Name,N_items,N_pages
0,COLECCIÓN,337,6
1,NOVEDADES,186,4
2,Outlet,345,6
3,Vestidos,71,2
4,Faldas,17,1
5,Pantalones,31,1
6,Jeans,3,1
7,Chaquetas y Rebecas,15,1
8,Camisetas y Camisas,45,1
9,Jerseis,10,1


In [0]:
from time import sleep
from random import randint

item_list = []
col_names = ["Name", "Color", "Item_URL", "Price", "Section", "Description", "Status_1", "Status_2", "Status_3"]
item_list.append(col_names)

# cambiamos los headers (user-agent) para evitar ser bloqueados por el sitio
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
"Accept":"text/html,application/xhtml+xml,application/xml; q=0.9,image/webp,*/*;q=0.8"}


def item_scraper(url,url_temp):
  '''
  Función que, dada una url de sección y una URL relativa a la página dentro de la sección,
  obtiene datos de todos los artículos.
  '''
  # obtenemos la seccion asociada a la URL
  seccion = dict_url_section[url]
  
  # hacemos el request en la url de la página dentro de la sección
  page = requests.get(url_temp, headers = headers)
  soup = BeautifulSoup(page.content, 'lxml')
  
  # excepción que previene las secciones en las que solo hay una página para navegar
  # y no se puede obtener el número total de páginas (dicho número es 1)
  try:
    soup2 = soup.find("ul",{"class": "page-list clearfix text-sm-center"})
    n_pages = soup2.find_all("a")[-2].get_text().strip() # número de páginas
  except:
    # sección dónde solo hay una página para navegar
    n_pages = 1
  
  # excepción que previene las secciones donde no hay articles y el método falla
  try:
    products = soup.find("div", {"class":"products row"})
    for article in products.find_all("article"): # cada uno de los artículos

      description = article.find("div",{'class': "product-description"})
      name = description.find("h2").a.get_text().split()[:-1] 
      name = ' '.join(name) # nombre del articulo
      color = description.find("h2").a.get_text().split()[-1] # color

      url_article = description.find("h2").a.attrs['href'] # url del artículo


      price_shipping = article.find("div", {"class": "product-price-and-shipping"})
      price = price_shipping.find("span", {"class": "price"}).text # precio

      item = [name, color, url_article, price]

      item.append(seccion.upper())  # sección a la que pertenece el artículo
      
      #sleep(randint(2,4))

      # item scraper: para obtener la descripción de cada producto
      item_page = requests.get(url_article, headers = headers)
      item_soup = BeautifulSoup(item_page.content, 'lxml')
      info = item_soup.find('div',{'class':'product-information'})
      description = info.div.get_text() # texto con la descripción del producto
      item.append(description)
      item_list.append(item)
      
      
      # Estados especiales de los artículos  
      flags = article.find("ul", {"class": "product-flags"})
      for status in flags.find_all('li'):
        st = status.get_text().replace('\n', ' ').strip()
        item.append(st)
      
  except:
      "seccion donde no hay articulos"
  
  # obtenemos el listado de artículos
  return(item_list)

In [5]:
# aplicamos la función anterior a todas las secciones y las páginas dentro de las mismas
req_count = 0
from time import time
start_time = time()

for url,sect in dict_url_section.items():
  print('---------')
  # páginas dentro de la sección. dict_pages[url] nos devuelve el número de páginas que tiene la sección
  for i in range(1, int( float(dict_pages[url]) ) + 1 ): 
    url_temp = url # url base de subseccion
    url_temp += '?page=' # añadimos el string necesario para pasar de pagina
    url_temp += str(i)
    
    item_scraper(url,url_temp)
    req_count += 1
    elapsed_time = time() - start_time
    print('Obteniendo datos de la sección {}. Se han visitado {} seccion(es) en {:5.2f} segundos.'.format(sect, req_count, elapsed_time))


---------
Obteniendo datos de la sección COLECCIÓN. Se han visitado 1 seccion(es) en 80.84 segundos.
Obteniendo datos de la sección COLECCIÓN. Se han visitado 2 seccion(es) en 151.33 segundos.
Obteniendo datos de la sección COLECCIÓN. Se han visitado 3 seccion(es) en 224.18 segundos.
Obteniendo datos de la sección COLECCIÓN. Se han visitado 4 seccion(es) en 298.72 segundos.
Obteniendo datos de la sección COLECCIÓN. Se han visitado 5 seccion(es) en 372.17 segundos.
Obteniendo datos de la sección COLECCIÓN. Se han visitado 6 seccion(es) en 419.97 segundos.
---------
Obteniendo datos de la sección NOVEDADES. Se han visitado 7 seccion(es) en 493.29 segundos.
Obteniendo datos de la sección NOVEDADES. Se han visitado 8 seccion(es) en 594.83 segundos.
Obteniendo datos de la sección NOVEDADES. Se han visitado 9 seccion(es) en 761.05 segundos.
Obteniendo datos de la sección NOVEDADES. Se han visitado 10 seccion(es) en 782.37 segundos.
---------
Obteniendo datos de la sección Outlet. Se han visi

In [6]:
## construimos un dataframe con los datos obtenidos: articulos

import pandas as pd
df = pd.DataFrame(item_list)

col_names = df.iloc[0]
items_df  = pd.DataFrame(df.values[1:], columns = col_names)

items_df

,Name,Color,Item_URL,Price,Section,Description,Status_1,Status_2,Status_3
0,Blusa Nigi,BLANCO,https://latiendadevalentina.com/low-cost/blusa...,"20,99 €",COLECCIÓN,Blusa de manga corta con escote de pico. Tejid...,Novedades,None,None
1,Camiseta Algodón Pati,MARINO,https://latiendadevalentina.com/low-cost/camis...,"15,99 €",COLECCIÓN,Camiseta de algodón con tirantes anchos y cuel...,Novedades,Básicos,None
2,Camiseta Algodón Pati,BEIGE,https://latiendadevalentina.com/low-cost/camis...,"15,99 €",COLECCIÓN,Camiseta de algodón con tirantes anchos y cuel...,Novedades,Básicos,None
3,Camiseta Algodón Pati,NEGRO,https://latiendadevalentina.com/low-cost/camis...,"15,99 €",COLECCIÓN,Camiseta de algodón con tirantes anchos y cuel...,Novedades,Básicos,None
4,Vestido Tencel Camis,MSTZA,https://latiendadevalentina.com/low-cost/vesti...,"26,99 €",COLECCIÓN,Vestido midi camisero de manga corta en tono m...,Novedades,None,None
5,Vestido Arat,MARINO,https://latiendadevalentina.com/low-cost/vesti...,"22,99 €",COLECCIÓN,Vestido recto de manga corta en tono marino. T...,Novedades,None,None
6,Vestido Tencel Camis,CORAL,https://latiendadevalentina.com/low-cost/vesti...,"26,99 €",COLECCIÓN,Vestido midi camisero de manga corta en tono c...,Novedades,None,None
7,Top Keo Estampado,ROSA,https://latiendadevalentina.com/low-cost/top-k...,"17,99 €",COLECCIÓN,Top de tirante fino con escote de pico. Tejido...,Novedades,None,None
8,Top Keo Estampado,Flores,https://latiendadevalentina.com/low-cost/top-k...,"17,99 €",COLECCIÓN,Top de tirante fino con escote de pico. Tejido...,Novedades,None,None
9,Vestido Azar Estamp,ROSA,https://latiendadevalentina.com/low-cost/vesti...,"25,99 €",COLECCIÓN,Vestido de manga corta con estampado de flores...,Novedades,None,None


In [0]:
# Creamos el fichero: secciones.csv
with open("secciones.csv", 'w') as csv_file:
    wr = csv.writer(csv_file, delimiter=',',quoting=csv.QUOTE_NONNUMERIC)
    for sect in section_list:
        wr.writerow(list(sect))

from google.colab import files
files.download("secciones.csv")

In [0]:
# Creamos el fichero: articulos.csv
with open("articulos.csv", 'w') as csv_file:
    wr = csv.writer(csv_file, delimiter=',',quoting=csv.QUOTE_NONNUMERIC)
    for art in item_list:
        wr.writerow(list(art))

from google.colab import files
files.download("articulos.csv")